In [1]:
import os
import numpy as np
from scipy.ndimage import rotate
from tqdm import tqdm
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation as R

In [49]:
dataset = 'UkBioBank'
side = 'L'

arrs = np.load(f'/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/S.C.-sylv./mask/Lskeleton.npy')
foldlabels = np.load(f'/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/S.C.-sylv./mask/Llabel.npy')
distbottoms = np.load(f'/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/S.C.-sylv./mask/Ldistbottom.npy')

In [50]:
dict_arrs = {'skeleton': arrs,
             'foldlabel': foldlabels,
             'distbottoms': distbottoms}

In [63]:
t = np.stack((arrs, foldlabels, distbottoms), axis=4)

In [64]:
t.shape

(21045, 42, 34, 49, 3, 1)

In [51]:
arrs.shape

(21045, 42, 34, 49, 1)

In [52]:
# given two vectors, compute the rotations to map one to the other
def rotation(v1, v2):
    """
    Compute a matrix R that rotates v1 to align with v2.
    v1 and v2 must be length-3 1d numpy arrays.
    """
    # unit vectors
    u = v1 / np.linalg.norm(v1)
    Ru = v2 / np.linalg.norm(v2)
    # dimension of the space and identity
    dim = u.size
    I = np.identity(dim)
    # the cos angle between the vectors
    c = np.dot(u, Ru)
    # a small number
    eps = 1.0e-10
    if np.abs(c - 1.0) < eps:
        # same direction
        return I
    elif np.abs(c + 1.0) < eps:
        # opposite direction
        return -I
    else:
        # the cross product matrix of a vector to rotate around
        K = np.outer(Ru, u) - np.outer(u, Ru)
        # Rodrigues' formula
        return I + K + (K @ K) / (1 + c)
    
# remove zeros function
def trim_zeros(arr):
    """Returns a trimmed view of an n-D array excluding any outer
    regions which contain only zeros.
    """
    slices = tuple(slice(idx.min(), idx.max() + 1) for idx in np.nonzero(arr))
    return arr[slices]

In [53]:
a = [1, 0, 0]
b = [0.534, -0.293, 0.793] # TODO: HERE DEFINE THE NORMAL TO HULL
b = b / np.linalg.norm(b)

rot = rotation(b,a)
print(rot)
print(np.dot(rot, b))

r = R.from_matrix(rot)
angles = r.as_euler('zyx', degrees=True)
print(angles)
angles = [-14.4, 28.75, 52.5] # TODO: is it the right order ??? shift from 1 to the left compared to what is given by normal to hull ?
# when angle is defined with morphologist, still need this reordering ?

[[ 0.53403899 -0.29302139  0.7930579 ]
 [ 0.29302139  0.9440291   0.15148437]
 [-0.7930579   0.15148437  0.59000988]]
[1.00000000e+00 1.26288993e-17 6.75722575e-17]
[ 28.75309824  52.47220192 -14.39959725]


In [54]:
dict_rot_arrs = {}

axes = [(0, 1), (0, 2), (1, 2)]
for key, arrs in dict_arrs.items():
    print(f'ARRAY TYPE : {key}')
    if key=='distbottoms':
        cval=32501
    else:
        cval=0
    arr_list = []
    print(f'Non zero voxels before rotation : {np.sum(arrs!=cval)}')
    for arr in tqdm(arrs):
        arr = arr[:,:,:,0]
        for ax, angle in zip(axes, angles):
            rot_arr = rotate(arr,
                                angle=angle,
                                axes=ax,
                                order=0,
                                reshape=True,
                                mode='constant',
                                cval=cval)
        arr_list.append(rot_arr)

    rot_arrs = np.stack(arr_list)
    print(f'Non zero voxels after rotation : {np.sum(rot_arrs!=cval)}')
    print(f'Dimension after rotation : {rot_arrs.shape}')
    if key=='distbottoms':
        # set 0 to vx to remove (temporarily)
        rot_arrs[rot_arrs==0]=-1
        rot_arrs[rot_arrs>=32500]=0
    rot_arrs_trimmed = trim_zeros(rot_arrs)
    if key=='distbottoms':
        rot_arrs_trimmed[rot_arrs_trimmed==0]=32501
        rot_arrs_trimmed[rot_arrs_trimmed==-1]=0
    print(f'Dimension after trimming : {rot_arrs_trimmed.shape}')
    # volume ration should be < 1 !
    print(f"volume ratio after / before rotation : {np.prod(rot_arrs_trimmed.shape[1:]) / np.prod(arrs.shape[1:])}")
    # are topological values affected ?
    print(np.unique(arrs, return_counts=True))
    print(np.unique(rot_arrs_trimmed, return_counts=True))

    dict_rot_arrs[key]=rot_arrs_trimmed

ARRAY TYPE : skeleton
Non zero voxels before rotation : 23430916


100%|██████████| 21045/21045 [04:59<00:00, 70.32it/s]


Non zero voxels after rotation : 23368337
Dimension after rotation : (21045, 42, 60, 57)
Dimension after trimming : (21045, 40, 53, 24)
volume ratio after / before rotation : 0.727148002057966
(array([ 0, 10, 30, 35, 40, 50, 60, 70, 80, 90], dtype=int16), array([1449129824,        665,    3408739,    2759990,     156026,
             1923,   16692492,       6078,     404983,         20]))
(array([ 0, 10, 30, 35, 40, 50, 60, 70, 80, 90], dtype=int16), array([1047401263,        679,    3397003,    2752904,     155577,
             1885,   16651653,       6072,     402540,         24]))
ARRAY TYPE : foldlabel
Non zero voxels before rotation : 23430916


100%|██████████| 21045/21045 [04:53<00:00, 71.66it/s]


Non zero voxels after rotation : 23368337
Dimension after rotation : (21045, 42, 60, 57)
Dimension after trimming : (21045, 40, 53, 24)
volume ratio after / before rotation : 0.727148002057966
(array([-28014,      0,      1, ...,   7700,   7709,  17733], dtype=int16), array([         1, 1449129824,        586, ...,          6,          3,
                2]))
(array([-28014,      0,      1, ...,   7700,   7709,  17733], dtype=int16), array([         1, 1047401263,        586, ...,          4,          2,
                2]))
ARRAY TYPE : distbottoms
Non zero voxels before rotation : 23430916


100%|██████████| 21045/21045 [04:48<00:00, 73.07it/s]


Non zero voxels after rotation : 23368337
Dimension after rotation : (21045, 42, 60, 57)
Dimension after trimming : (21045, 40, 53, 24)
volume ratio after / before rotation : 0.727148002057966
(array([    0,   100,   141,   173,   200,   241,   273,   282,   300,
         314,   341,   346,   373,   382,   400,   414,   423,   441,
         446,   455,   473,   482,   487,   500,   514,   519,   523,
         541,   546,   555,   564,   573,   582,   587,   596,   600,
         614,   619,   623,   628,   641,   646,   655,   660,   664,
         673,   682,   687,   692,   696,   700,   705,   714,   719,
         723,   728,   737,   741,   746,   755,   760,   764,   769,
         773,   782,   787,   792,   796,   800,   801,   805,   814,
         819,   823,   828,   833,   837,   841,   846,   855,   860,
         864,   865,   869,   873,   878,   882,   887,   892,   896,
         900,   901,   905,   910,   914,   919,   923,   928,   933,
         937,   941,   942,   946,  

In [58]:
rot_arrs = dict_rot_arrs['skeleton']
rot_foldlabel = dict_rot_arrs['foldlabel']
rot_distbottom = dict_rot_arrs['distbottoms']

In [59]:
# save as np


array([[[[32501, 32501, 32501, ..., 32501, 32501, 32501],
         [32501, 32501, 32501, ..., 32501, 32501, 32501],
         [32501, 32501, 32501, ..., 32501, 32501, 32501],
         ...,
         [32501, 32501, 32501, ..., 32501, 32501, 32501],
         [32501, 32501, 32501, ..., 32501, 32501, 32501],
         [32501, 32501, 32501, ..., 32501, 32501, 32501]],

        [[32501, 32501, 32501, ..., 32501, 32501, 32501],
         [32501, 32501, 32501, ..., 32501, 32501, 32501],
         [32501, 32501, 32501, ..., 32501, 32501, 32501],
         ...,
         [32501, 32501, 32501, ..., 32501, 32501, 32501],
         [32501, 32501, 32501, ..., 32501, 32501, 32501],
         [32501, 32501, 32501, ..., 32501, 32501, 32501]],

        [[32501, 32501, 32501, ..., 32501, 32501, 32501],
         [32501, 32501, 32501, ..., 32501, 32501, 32501],
         [32501, 32501, 32501, ..., 32501, 32501, 32501],
         ...,
         [32501, 32501, 32501, ..., 32501, 32501, 32501],
         [32501, 32501, 32